# **_Chess Piece Classification_**

### __*This CNN is supposed to determine what kind of Chess Piece is in the Image*__

<br>

### __*Import Libraries and choose the device*__

In [176]:
# Import of the Libraries
import os
import csv
import numpy as np
import pandas as pd
import skimage
import matplotlib.pyplot as plt

# Torch libraries
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Device config (Pick your set-up)
GPU = torch.device('cpu')  # NVIDIA GPUs
# GPU = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # NVIDIA GPUs
# GPU = torch.device('mps' if torch.has_mps else 'cpu')  # ARM GPUs (M1, M2, ...)
print('Using the Processor') if GPU == torch.device('cpu') else print('Using the Graphics Card')

Using the Processor


<br>

### __*Create the .csv file for Dataset*__

In [177]:
def create_csv(csv_path, file_path, rewrite=False):
    if not os.path.exists(csv_path) or rewrite:
        file = open(csv_path, 'w', newline='')  # Create the file
        writer = csv.writer(file)               # Create the writer
        categories = os.listdir(FILE_PATH)      # Load all categories
        translate = dict()                      # Class : Number

        # Create a .csv file of all images & their class
        for idx in range(len(categories)):
            translate[idx] = [categories[idx]]
            tmp_path = os.path.join(file_path, categories[idx])
            tmp_images = os.listdir(tmp_path)
            for img in tmp_images:
                img_path = os.path.join(categories[idx], img)  # Sub-File + img
                writer.writerow((img_path, idx))  # Turn category into index

        file.close()                            # Close the file
        return translate

<br>

### __*Convert the Images into a Dataset*__

In [178]:
class ChessDataset(Dataset):

    def __init__(self, csv_path, file_path, transform=None):
        self.annotations = pd.read_csv(csv_path)
        self.file_path = file_path
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.file_path, self.annotations.iloc[index, 0])
        image = skimage.io.imread(img_path)
        y_label = torch.tensor(self.annotations.iloc[index, 1])

        if self.transform:
            image = self.transform(image)

        return image, y_label

<br>

### __*Convolutional Neural Network*__

In [179]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # input = 3 colour channels, output = 6 (our choice), filter = 4*4 (our choice)
        self.conv1 = nn.Conv2d(3, 6, 4)
        self.pool = nn.MaxPool2d(2, 2)  # take 2*2, then move 2 px
        # input = 6 channels, output = 16 (our choice), filter = 4*4 (our choice)
        self.conv2 = nn.Conv2d(6, 16, 4)
        self.fc1 = nn.Linear(16 * 19 * 19, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 6)


    def forward(self, x):
        # -> n, 3, 85, 85
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 19 * 19)                            # -> n, 400
        x = torch.nn.functional.relu(self.fc1(x))               # -> n, 120
        x = torch.nn.functional.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                                         # -> n, 10
        return x

<br>

### __*Training Preparation*__

In [180]:
# Image Dimensions
IMG_SIZE = 85  # 85*85 pixels

# Define the directory
FILE_PATH = './data/chess/pieces'   # Sub-Folders Location
CSV_PATH = './data/chess/data.csv'  # CSV Location
CATEGORIES = os.listdir(FILE_PATH)  # Turn Sub-Folder names into a list

# Hyper-parameters
num_epochs = 5
batch_size = 1
learning_rate = 0.001

# Create the .csv file & dictionary
indexing = create_csv(CSV_PATH, FILE_PATH, rewrite=True)
print(f'Translate: {indexing}')

# Create the Datasets & DataLoaders
dataset = ChessDataset(CSV_PATH, FILE_PATH, torchvision.transforms.ToTensor())
n_80 = int(len(dataset) / 100 * 80)  # 80% for training
n_20 = int(len(dataset) - n_80)      # 20% for testing
print(f'N. of images: {len(dataset)} | Training: {n_80} | Testing: {n_20}')

train_dataset, test_dataset = torch.utils.data.random_split(dataset,[n_80, n_20])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

# Create the Model
model = ConvNet().to(GPU)

# Choose type of Loss & Optimization function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


def imshow(img):
    img = img / 2 + 0.5  # un-normalize
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1, 2, 0)))
    plt.show()


# Get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)
print(images.shape)

# Show images
imshow(torchvision.utils.make_grid(images))

Translate: {0: ['pawn'], 1: ['queen'], 2: ['rook'], 3: ['bishop'], 4: ['king'], 5: ['knight']}
N. of images: 516 | Training: 412 | Testing: 104


<br>

### __*Training Loop*__

In [181]:
# Variables for
n_total_steps = len(train_loader)
mean_loss = 0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(GPU)
        labels = labels.to(GPU)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        mean_loss += loss.item()

        # Backward & optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch {epoch + 1} / {num_epochs} | Step {i+1} / {n_total_steps} | '
                  f'Loss: {(mean_loss/2000):.4f}')
            mean_loss = 0

print('--- Training Finished ---')
PATH = './chess.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(6)]
    n_class_samples = [0 for i in range(6)]
    for images, labels in test_loader:
        images = images.to(GPU)
        labels = labels.to(GPU)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    accuracy = 100.0 * n_correct / n_samples
    print(f'Accuracy of the Model: {accuracy} %')

    for i in range(6):
        accuracy = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {CATEGORIES[i]}: {accuracy} %')

TypeError: img should be PIL Image. Got <class 'numpy.ndarray'>